In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<em>the code of this notebook is hidden for easier reading.
to toggle it on/off click <a href="javascript:code_toggle()">here</a>.</em>''')

### PANORAMAS OF CINEMA online

# // TEXT SEARCH
v0.00

# 1. SEARCH THE VIDEOTHEQUE

### > type your query and options

In [2]:
# GLOBAL
app_http = 'http://ec2-54-154-19-72.eu-west-1.compute.amazonaws.com:8080'
button_color = 'black' # #7d1b27

# DISPLAY WIDGETS 
from IPython.display import display, clear_output
from ipywidgets import interactive, interact
import ipywidgets as widgets

def search_text(text, min_conf, max_conf, max_results, text_model):
    return

w_search_text = interactive(search_text, text='gone south',
                            min_conf=widgets.FloatSlider(min=0.01, max=1., step=.001, value=.0),
                            max_conf=widgets.FloatSlider(min=0.01, max=1., step=.001, value=1.),
                            max_results='10',
                            text_model=widgets.RadioButtons( options=['BOW','TFIDF'], value='TFIDF', description='model' )
                           )

display(w_search_text)


interactive(children=(Text(value='gone south', description='text'), FloatSlider(value=0.01, description='min_c…

In [3]:
# REFRESH QUERY
import requests, ast

def format_text(query):
    this_query = query.replace(' ', '%20')
    return this_query
    
def refresh_query(event):
    clear_output()
    display(search_button)
    
    text = format_text(w_search_text.kwargs['text'])
    confX = w_search_text.kwargs['min_conf']
    confY = w_search_text.kwargs['max_conf']
    model = w_search_text.kwargs['text_model']
    max_results = w_search_text.kwargs['max_results']

    # 
    this_query = app_http+'/?'
    if text: this_query = this_query+'text=%s&confidenceX=%s&confidenceY=%s'%(text,confX,confY)
    if model: this_query = this_query+'&model=%s'%model
    if max_results: this_query = this_query+'&maxResults=%s'%max_results
    #print(this_query)
    
    global my_text
    my_text = ast.literal_eval(requests.get(this_query).text)['body']
    #print (len(my_text), 'images with this search') # this_query[73:]
    #print(my_text)
    
    for movie, prev, this, nex in my_text:
        print(movie)
        print(prev)
        print(this)
        print(nex)
        print()
    
    #global my_globals
    #my_globals = {'plotted':[], 'w_buttons':[]}

                    
search_button = widgets.Button(description="search", button_style='info')
search_button.style.button_color = button_color
display(search_button)
search_button.on_click(refresh_query)

Button(button_style='info', description='search', style=ButtonStyle(button_color='black'))